In [1]:
from utils import dataset_manager, model_manager
import os
from flask import Flask, request, jsonify
from routes.interact_with_csv_files import csv_routes
from deep_translator import GoogleTranslator
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data_dir = '/Users/VoThinhPhat/Desktop/data'
dataset_manager = dataset_manager.Dataset(data_dir=data_dir)

Finish L05_V031 with frame_id 35869...

In [3]:
# clip vit large patch14 model
model_clip14 = model_manager.CLIP_14_model()

# remember to download model_config_file and model_file (contact vtphatt2 for link)
model_config_file = os.path.join(os.getcwd(), 'task-former', 'code', 'training', 
                                 'model_configs', 'ViT-B-16.json')
model_file = os.path.join(os.getcwd(), 'task-former', 'model', 'tsbir_model_final.pt')
model_task_former = model_manager.TASK_former_model(model_config_file=model_config_file,
                                                    model_file=model_file)

Model openai-clip-vit-large-patch14 is using mps


/Users/VoThinhPhat/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Model TASK-former is using mps


/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/src/utils/model_manager.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_file, map_locatio

In [4]:
def searchByText(text_query, k = 200):
    submission_list = []
    text_embedding = [model_clip14.inference(text_query)]

    dataset = dataset_manager.get_dataset()
    result = []
    for video_name, embeddings_array in dataset_manager.get_video_clip14_embedding_dict().items():
        sim_scores = cosine_similarity(text_embedding, embeddings_array).flatten()
        for index, score in enumerate(sim_scores):
            result.append((video_name, index, score))

    result.sort(key=lambda item: item[2], reverse=True)

    visited = [False] * k
    for i in range(0, k):
        if (not visited[i]):
            visited[i] = True
            video_name = result[i][0]
            x = [video_name, [(dataset[video_name][result[i][1]]['filepath'], dataset[video_name][result[i][1]]['frame_id'])]]

            for j in range(i + 1, k):
                if (not visited[j] and video_name == result[j][0] and abs(result[i][1] - result[j][1]) < 8):
                    x[1].append((dataset[video_name][result[j][1]]['filepath'], dataset[video_name][result[j][1]]['frame_id']))
                    visited[j] = True
            
            x[1] = sorted(x[1], key=lambda a:a[1])
            submission_list.append(x)
    
    print(submission_list)

    return submission_list

In [5]:
# def temporalSearch(text_first_this, text_then_that, k = 100, range_size = 8):
#     submission_list = []

#     video_range = dataset_manager.get_video_range()
#     image_samples = dataset_manager.get_image_samples()

#     x = model_clip14.inference(text_first_this)
#     y = model_clip14.inference(text_then_that)

#     results = []
#     for video_name in video_range.keys():
#         low = video_range[video_name][0] 
#         high = video_range[video_name][1] + 1 
#         vectors = []
#         for i in range(low, high):
#             vectors.append(image_samples[i]['clip-14'])
#         num_vectors = len(vectors)

#         for i in range(0, num_vectors - range_size + 1, 10):
#             block = vectors[i:i+range_size]
#             x_cos_sim = cosine_similarity([x], block[:int(0.7 * range_size)])[0]
#             y_cos_sim = cosine_similarity([y], block[int(0.3 * range_size):])[0]
#             block_similarity = (np.max(x_cos_sim) + np.max(y_cos_sim)) / 2

#             results.append((block_similarity, video_name, i))

#     results.sort(key=lambda x: x[0], reverse=True)
#     top_results = results[:k]

#     for similarity, video_name, best_index in top_results:
#         video_samples = []
#         low = video_range[video_name][0] 
#         for j in range(best_index, best_index + range_size):
#             sample = image_samples[low + j]
#             video_samples.append((sample['filepath'], int(sample['frame_id'])))
#         submission_list.append([video_name, video_samples])
    
#     return submission_list

In [6]:
app = Flask(__name__)

# Register the blueprint with the main app
app.register_blueprint(csv_routes)

@app.route('/search_by_text', methods=['POST'])
def search_by_text():
    data = request.json
    search_text = data.get('searchText')
    translated_text = GoogleTranslator(source='vi', target='en').translate(search_text)
    submission_list = searchByText(translated_text, k=100)  # Ensure this returns an ordered dict if necessary

    response = jsonify({
        "translated_text": translated_text,
        "submission_list": submission_list  # Use a list instead of a dict
    })
    response.headers['Cache-Control'] = 'no-store, no-cache, must-revalidate, max-age=0'
    response.headers['Pragma'] = 'no-cache'
    response.headers['Expires'] = '0'

    return response, 200

# @app.route('/temporal_search', methods=['POST'])
# def temporal_search():
#     # Extract the request data
#     data = request.json
#     text_first_this = data.get('textFirstThis')
#     text_then_that = data.get('textThenThat')
#     translated_first_this = GoogleTranslator(source='vi', target='en').translate(text_first_this)
#     translated_then_that = GoogleTranslator(source='vi', target='en').translate(text_then_that)

#     submission_list = temporalSearch(translated_first_this, translated_then_that, k = 30, range_size=30)

#     # Prepare and return the response
#     response = jsonify({
#         "translated_first_this": translated_first_this,
#         "translated_then_that": translated_then_that,
#         "submission_list": submission_list
#     })
#     response.headers['Cache-Control'] = 'no-store, no-cache, must-revalidate, max-age=0'
#     response.headers['Pragma'] = 'no-cache'
#     response.headers['Expires'] = '0'

#     return response, 200

app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Sep/2024 01:14:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2024 01:14:19] "GET /static/canvasDrawing.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 01:14:19] "GET /static/search_mode.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 01:14:19] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 01:14:19] "GET /static/reload.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 01:14:19] "GET /static/zoom.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 01:14:19] "GET /static/temporal_search.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 01:14:19] "GET /static/interact_with_csv_files.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 01:14:19] "GET /image/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L05/L05_V005/0586.jpg HTTP/1.1" 404 -
127.0.0.1 - - [27/Sep/2024 01:14:19] "GET /image/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L05/L05_V005/0587.jpg HTTP/1.1" 404 -
127.0.0.1 - - [27/Sep/20

[['L05_V005', [('/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L05/L05_V005/19371.jpg', '19371'), ('/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L05/L05_V005/19391.jpg', '19391'), ('/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L05/L05_V005/19410.jpg', '19410')]], ['L01_V031', [('/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L01/L01_V031/21947.jpg', '21947'), ('/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L01/L01_V031/21971.jpg', '21971'), ('/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L01/L01_V031/21994.jpg', '21994')]], ['L05_V005', [('/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L05/L05_V005/17084.jpg', '17084'), ('/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L05/L05_V005/17101.jpg', '17101'), ('/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L05/L05_V005/17117.jpg', '17117')]], ['L01_V030', [('/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L01/L01_V030

127.0.0.1 - - [27/Sep/2024 01:14:29] "GET /image/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L01/L01_V031/06444.jpg HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2024 01:14:29] "GET /image/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L01/L01_V031/06416.jpg HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2024 01:14:29] "GET /image/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L04/L04_V026/09483.jpg HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2024 01:14:29] "GET /image/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L04/L04_V026/09497.jpg HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2024 01:14:29] "GET /image/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L04/L04_V026/09510.jpg HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2024 01:14:29] "GET /image/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L03/L03_V011/22402.jpg HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2024 01:14:29] "GET /image/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L03/L03_V011/2243